In [75]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense

from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [76]:
data = pd.read_csv("dataset.csv")
data.head()

,Name,2023-05-09 00:00:00-03:00,2023-05-10 00:00:00-03:00,2023-05-11 00:00:00-03:00,2023-05-12 00:00:00-03:00,2023-05-15 00:00:00-03:00,2023-05-16 00:00:00-03:00,2023-05-17 00:00:00-03:00,2023-05-18 00:00:00-03:00,2023-05-19 00:00:00-03:00,...,2023-05-25 00:00:00-03:00,2023-05-26 00:00:00-03:00,2023-05-29 00:00:00-03:00,2023-05-30 00:00:00-03:00,2023-05-31 00:00:00-03:00,2023-06-01 00:00:00-03:00,2023-06-02 00:00:00-03:00,2023-06-05 00:00:00-03:00,2023-06-06 00:00:00-03:00,2023-06-07 00:00:00-03:00
0,USIM5.SA,7.600000,7.630000,7.440000,7.460000,7.610000,7.470000,7.720000,7.80,7.990000,...,7.280000,7.310000,7.280000,7.070000,7.020000,7.160000,7.380000,7.410000,7.390000,7.340000
1,MULT3.SA,27.030001,27.200001,27.100000,26.850000,26.920000,26.440001,26.620001,26.92,27.049999,...,26.790001,26.920000,26.790001,26.750000,26.530001,26.480000,26.129999,25.990000,26.570000,27.080000
2,BBSE3.SA,33.709999,33.799999,33.970001,34.160000,32.139999,32.150002,31.969999,31.32,30.900000,...,31.809999,31.900000,32.090000,31.690001,30.930000,30.520000,30.990000,31.040001,30.990000,30.990000
3,JBSS3.SA,17.790001,17.299999,17.299999,16.139999,16.170000,16.219999,16.200001,16.40,17.000000,...,16.430000,16.400000,16.410000,16.410000,16.799999,16.410000,16.770000,16.879999,17.620001,17.660000
4,BRKM5.SA,26.280001,25.790001,25.559999,24.320000,22.690001,22.760000,23.370001,23.42,24.180000,...,23.040001,22.969999,22.889999,22.700001,22.459999,23.030001,23.850000,24.100000,24.850000,25.450001


In [77]:
def normalize_by_row(df):
    # normalizar por coluna para evitar o impacto da diferença entre as ações
    df = df.subtract(df.min(axis=1).to_numpy(), axis=0)
    df = df.divide(df.max(axis=1).to_numpy(), axis=0)
    return df

normalized_data = normalize_by_row(data.iloc[:, 1:])

X = normalized_data.iloc[:, :-1]
y = normalized_data.iloc[:, -1]

X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.6)
X_test, X_val, y_test, y_val = train_test_split(X_rem, y_rem, test_size=0.5)

X_train, y_train = X_train.to_numpy(), y_train.to_numpy()
X_test, y_test = X_test.to_numpy(), y_test.to_numpy()


In [70]:
normalized_data#.iloc[2]
dt = data.iloc[:, 1:]
dt = dt.subtract(dt.min(axis=1).to_numpy(), axis=0)
dt = dt.divide(dt.max(axis=1).to_numpy(), axis=0)

In [78]:
dt

,2023-05-09 00:00:00-03:00,2023-05-10 00:00:00-03:00,2023-05-11 00:00:00-03:00,2023-05-12 00:00:00-03:00,2023-05-15 00:00:00-03:00,2023-05-16 00:00:00-03:00,2023-05-17 00:00:00-03:00,2023-05-18 00:00:00-03:00,2023-05-19 00:00:00-03:00,2023-05-22 00:00:00-03:00,...,2023-05-25 00:00:00-03:00,2023-05-26 00:00:00-03:00,2023-05-29 00:00:00-03:00,2023-05-30 00:00:00-03:00,2023-05-31 00:00:00-03:00,2023-06-01 00:00:00-03:00,2023-06-02 00:00:00-03:00,2023-06-05 00:00:00-03:00,2023-06-06 00:00:00-03:00,2023-06-07 00:00:00-03:00
0,0.597938,0.628866,0.432990,0.453608,0.608248,0.463917,0.721649,0.804124,1.000000,0.845361,...,0.268042,0.298969,0.268042,0.051547,0.000000,0.144330,0.371134,0.402062,0.381443,0.329897
1,0.859504,1.000000,0.917355,0.710744,0.768595,0.371901,0.520662,0.768595,0.876032,0.719008,...,0.661157,0.768595,0.661157,0.628099,0.446281,0.404958,0.115702,0.000000,0.479338,0.900826
2,0.876373,0.901099,0.947803,1.000000,0.445055,0.447803,0.398351,0.219780,0.104395,0.112637,...,0.354395,0.379121,0.431319,0.321429,0.112637,0.000000,0.129121,0.142857,0.129121,0.129121
3,1.000000,0.703030,0.703030,0.000000,0.018182,0.048485,0.036364,0.157576,0.521212,0.606060,...,0.175758,0.157576,0.163636,0.163636,0.400000,0.163636,0.381818,0.448484,0.896970,0.921212
4,1.000000,0.871728,0.811518,0.486911,0.060210,0.078534,0.238220,0.251309,0.450262,0.246073,...,0.151833,0.133508,0.112565,0.062828,0.000000,0.149215,0.363875,0.429320,0.625655,0.782723
5,0.063106,0.058253,0.000000,0.063106,0.504854,0.325243,0.441748,0.791262,1.000000,0.956311,...,0.674757,0.791262,0.660195,0.669903,0.543690,0.441748,0.635922,0.684466,0.800971,0.723301
6,0.140787,0.140787,0.020704,0.000000,0.107661,0.033126,0.372671,0.420290,0.536232,0.436853,...,0.554865,0.666667,0.594203,0.517598,0.544514,0.774327,0.755694,0.766046,0.890269,1.000000
7,0.197473,0.394947,0.620629,0.507787,0.620629,0.564206,0.648840,0.592418,0.423158,0.253893,...,0.705260,0.564206,0.535998,0.282105,0.000000,0.377707,0.971716,0.943429,0.943429,1.000000
8,0.603774,0.490566,0.396226,0.254717,0.396226,0.000000,0.273586,0.528302,0.622642,0.462265,...,0.462265,0.424529,0.330189,0.509435,0.320755,0.594339,0.867924,0.839623,1.000000,0.783019
9,0.000000,0.020833,0.159725,0.208337,0.375007,0.381952,0.333339,0.368062,0.631957,0.520843,...,0.659735,0.659735,0.673624,0.368062,0.236116,0.423618,0.436812,0.819223,1.000000,0.993047


In [81]:
# Reshape dos dados de treinamento para o formato esperado pela RNN
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Construção do modelo
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))

# Compilação do modelo
model.compile(loss='mean_squared_error', optimizer='adam')

# Treinamento do modelo
model.fit(X_train, y_train, epochs=200, batch_size=1, verbose=1)

# Reshape dos dados de teste para o formato esperado pela RNN
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Predição dos dados de teste
predictions = model.predict(X_test)

print("Predições:")
for i in range(len(predictions)):
    print("Entrada:", X_test[i].flatten())
    print("Valor real:", y_test[i])
    print("Predição:", predictions[i][0])
    print()

Epoch 1/200


2023-06-11 09:59:26.318531: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-11 09:59:26.320095: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-11 09:59:26.321011: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

10/10 [==============================] - 1s 6ms/step - loss: 0.4223
Epoch 2/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0983
Epoch 3/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0677
Epoch 4/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0711
Epoch 5/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0601
Epoch 6/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0716
Epoch 7/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0752
Epoch 8/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0667
Epoch 9/200
10/10 [==============================] - 0s 6ms/step - loss: 0.0664
Epoch 10/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0529
Epoch 11/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0520
Epoch 12/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0536
Epoch 13/200
10/10 [=============================

10/10 [==============================] - 0s 4ms/step - loss: 0.0051
Epoch 103/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0095
Epoch 104/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0208
Epoch 105/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0152
Epoch 106/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0068
Epoch 107/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0058
Epoch 108/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0055
Epoch 109/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0068
Epoch 110/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0055
Epoch 111/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0057
Epoch 112/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0053
Epoch 113/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0060
Epoch 114/200
10/10 [=========

2023-06-11 09:59:36.092672: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-11 09:59:36.094085: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-11 09:59:36.094965: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus